In [ ]:
# qiime2 activation

In [ ]:
source activate qiime2-2020.6

In [ ]:
# For Illumina, 16SV4V5 sequences are merged with 18SV9 and ITS2 sequences in the same fastq files
# Extraction of 16SV4V5 sequences will be performed first

In [ ]:
# Importation of dada2 ASV table with abundances in qiime2 artefact format

In [ ]:
cd ./illumina_mangroves_intrasites_final
mkdir ./qiime2

qiime tools import \
--input-path ./dada2/seqtabnochim_mangrove_16S_18S_ITS.fna \
--type 'FeatureData[Sequence]' \
--output-path ./qiime2/rep-seqs_mangrove_16S_18S_ITS.qza

In [ ]:
biom convert -i ./dada2/seqtabnochim_mangrove_16S_18S_ITS.txt -o ./qiime2/table_mangrove_16s_18s_its_final.biom --table-type="OTU table" --to-hdf5

qiime tools import \
--input-path ./qiime2/table_mangrove_16s_18s_its_final.biom \
--type 'FeatureTable[Frequency]' \
--output-path ./qiime2/table_mangrove_16s_18s_its_final.qza

In [ ]:
######################
# extraction 16SV4

cd ./qiime2

qiime feature-classifier extract-reads \
  --i-sequences ./rep-seqs_mangrove_16S_18S_ITS.qza \
  --p-f-primer GTGYCAGCMGCCGCGGTAA \
  --p-r-primer CCGYCAATTYMTTTRAGTTT\
  --p-identity 0.5 \ # flexible identity rate because of the several degenerated nucleotides in markers
  --p-n-jobs 32 \
  --o-reads /home/tony/alice/qiime2/rep-seqs_mangrove_16sV4.qza \

qiime feature-table filter-features \
  --i-table ./table_mangrove_16s_18s_its_final.qza  \
  --m-metadata-file ./rep-seqs_mangrove_16sV4.qza \
  --o-filtered-table ./table_mangrove_16sV4.qza

In [ ]:
# Make 97% consensus sequences (OTU) from ASVs

In [ ]:
# for 16SV4V5
qiime vsearch cluster-features-de-novo \
  --i-table table_mangrove_16sV4.qza \
  --i-sequences rep-seqs_mangrove_16sV4.qza \
  --p-perc-identity 0.97 \
  --p-threads 32 \
  --o-clustered-table table-97_mangrove-16S_final.qza \
  --o-clustered-sequences rep-seqs-97_Mangrove-16S_final.qza

In [5]:
# Export OTU table

In [ ]:
qiime tools export \
    --input-path table-97_mangrove-16S_final.qza \
    --output-path ./
    
biom convert \
    -i ./feature-table.biom \
    -o ./feature-table-97_mangrove-16S_final.tsv --to-tsv

# open the .tsv file with LibreOffice or Excel, select abundances, then Edition > Find and replace,
# Seek for ^[0-9] and replace it by & (tick "regular expressions")

In [ ]:
# Taxonomic assignation of 16SV4V5 sequences with the database SILVA 138.1-ssu-nr99 fitted to the approriate markers 515f-926r
# classifier must have been prepared before, see https://docs.qiime2.org/2023.5/tutorials/feature-classifier/

qiime feature-classifier classify-sklearn \
   --i-reads ./rep-seqs-97_Mangrove-18S_final.qza\
   --i-classifier /media/tony/DATA2/tax/SILVA_MANGROVE/Illumina/silva-138.1-ssu-nr99-515f926r-classifier.qza \
   --p-n-jobs -1 \
   --output-dir /home/tony/alice/Assignation

# export in .biom format
qiime tools export \
    --input-path ./OTU-16S-taxo/classification.qza \
    --output-path ./OTU-16S-taxo

In [7]:
# Then have been created :
# - an OTU table at 3% (sequences and read abundances after quality filters) [feature-table-97_mangrove-16S_final.tsv]
# - a table with OTU taxonomical assignations [taxonomy.tsv]

# User shall then merge them, strictly keeping the order of the lines.